In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

In [3]:
train = datasets.FashionMNIST('~/pytorch/F_MNIST_data', 
                              train = True, 
                              download = True, 
                              transform=transform)

test = datasets.FashionMNIST('~/pytorch/F_MNIST_data',
                            train = False,
                            download = True,
                            transform = transform)


Extracting /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw



Extracting /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw



Extracting /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw



Extracting /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/adetunji/pytorch/F_MNIST_data/FashionMNIST/raw
Processing...


/home/adetunji/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
trainset = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
testset = torch.utils.data.DataLoader(test, batch_size = 64, shuffle = True)

In [5]:
from torch import optim, nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import torch.nn.functional as F